In [ ]:
!git clone https://github.com/Q-Future/Q-Align.git
%cd Q-Align
!pip install -e ".[train]"
!pip install flash_attn --no-build-isolation

Cloning into 'Q-Align'...
remote: Enumerating objects: 745, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 745 (delta 3), reused 2 (delta 1), pack-reused 732
Receiving objects: 100% (745/745), 42.06 MiB | 14.97 MiB/s, done.
Resolving deltas: 100% (440/440), done.
Updating files: 100% (177/177), done.
/content/Q-Align
Obtaining file:///content/Q-Align
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2

In [ ]:
!pip install deepspeed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Copy JSON file
!cp /content/drive/MyDrive/Q-Align_data/train_split_1.json /content/Q-Align/playground/data/ft/

# Create directory for video data
!mkdir -p /content/video_data

# Copy video files
!cp /content/drive/MyDrive/Q-Align_data/*.mp4 /content/video_data/
# If you have GIF files, use this command as well:
# !cp /content/drive/MyDrive/Q-Align_data/*.gif /content/video_data/

In [ ]:
%%writefile scripts/my_videos_lora.sh
#!/bin/bash


deepspeed q_align/train/train_mem.py \
    --deepspeed ./scripts/zero3.json \
    --model_name_or_path q-future/one-align \
    --version v1 \
    --data_path playground/data/ft/train_split_1.json \
    --image_folder /content/video_data \
    --bf16 True \
    --output_dir ./checkpoints/q-align-my-videos-lora \
    --num_train_epochs 1 \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_step 4 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 50000 \
    --save_total_limit 1 \
    --learning_rate 2e-5 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --tf32 True \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --dataloader_num_workers 4 \
    --lazy_preprocess True

Writing scripts/my_videos_lora.sh


In [ ]:
import torch
import subprocess

def print_gpu_memory():
    print(f"Allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB")
    print(f"Cached:    {torch.cuda.memory_reserved()/1e9:.2f} GB")

    # Get the total memory from nvidia-smi
    result = subprocess.check_output(
        ['nvidia-smi', '--query-gpu=memory.total', '--format=csv,nounits,noheader']
    ).decode('ascii').strip()
    total_memory = int(result)

    print(f"Total:     {total_memory/1024:.2f} GB")

# Print initial GPU memory state
print("Initial GPU memory state:")
print_gpu_memory()

Initial GPU memory state:
Allocated: 0.00 GB
Cached:    0.00 GB
Total:     40.00 GB


In [ ]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

In [ ]:
%%writefile scripts/my_videos_lora.sh
#!/bin/bash

# Print GPU memory before starting
echo "GPU memory before starting:"
nvidia-smi

deepspeed q_align/train/train_mem.py \
    --deepspeed ./scripts/zero2.json \
    --model_name_or_path q-future/one-align \
    --version v1 \
    --data_path playground/data/ft/train_split_1.json \
    --image_folder /content/video_data \
    --bf16 True \
    --output_dir ./checkpoints/q-align-my-videos-lora \
    --num_train_epochs 1 \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 4 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 50000 \
    --save_total_limit 1 \
    --learning_rate 2e-5 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --tf32 True \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --dataloader_num_workers 4 \
    --lazy_preprocess True

# Print GPU memory after finishing
echo "GPU memory after finishing:"
nvidia-smi

Overwriting scripts/my_videos_lora.sh


In [ ]:
!chmod +x scripts/my_videos_lora.sh
!bash scripts/my_videos_lora.sh

GPU memory before starting:
Wed Jul 10 04:04:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              43W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+---------------------------------------